In [1]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [2]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests

In [ ]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time
from datetime import date, timedelta

id = "id1"

auto_trading_stock = AutoTradingBot(id=id, virtual=False)

symbol = '001040'  # CJ
start_date = date(2023, 1, 1)
end_date = date(2024, 1, 1)
initial_capital = 5_000_000
target_trade_value_krw = 1_000_000
buy_trading_logic = ['rsi_trading']
sell_trading_logic = ['rsi_trading']
interval = 'day'

# 예수금과 상태를 계속 누적 추적할 객체
accumulated_trading_history = {
    'initial_capital': initial_capital,
    'total_quantity': 0,
    'average_price': 0,
    'history': [],
    'buy_dates': [],
    'sell_dates': [],
    'realized_pnl': 0,
    'unrealized_pnl': 0,
    'realized_roi': 0,
    'unrealized_roi': 0,
    'buy_count': 0,
    'sell_count': 0,
}

# 시뮬 날짜 범위
date_range = pd.date_range(start=start_date, end=end_date)

for current_date in date_range:
    print(f"\n📅 {current_date.strftime('%Y-%m-%d')} 시뮬레이션 중")

    try:
        # 하루치 데이터로 시뮬레이션 수행
        daily_result = auto_trading_stock.whole_simulate_trading(
            symbol=symbol,
            start_date=current_date - timedelta(days=100),  # 지표용 데이터 확보
            end_date=current_date,
            target_trade_value_krw=target_trade_value_krw,
            buy_trading_logic=buy_trading_logic,
            sell_trading_logic=sell_trading_logic,
            interval=interval,
            buy_percentage=None,
            ohlc_mode='default',
            initial_capital=accumulated_trading_history['initial_capital']  # 계속 업데이트
        )

        # 누적 자산 정보 갱신
        if daily_result:
            # 거래 내역 붙이기
            accumulated_trading_history['history'].extend(daily_result['history'])
            accumulated_trading_history['buy_dates'].extend(daily_result['buy_dates'])
            accumulated_trading_history['sell_dates'].extend(daily_result['sell_dates'])
            accumulated_trading_history['buy_count'] += daily_result['buy_count']
            accumulated_trading_history['sell_count'] += daily_result['sell_count']
            accumulated_trading_history['realized_pnl'] += daily_result['realized_pnl']
            accumulated_trading_history['unrealized_pnl'] = daily_result['unrealized_pnl']
            accumulated_trading_history['realized_roi'] = daily_result['realized_roi']
            accumulated_trading_history['unrealized_roi'] = daily_result['unrealized_roi']
            accumulated_trading_history['initial_capital'] = daily_result['initial_capital']  # 예수금 반영
            accumulated_trading_history['total_quantity'] = daily_result['total_quantity']
            accumulated_trading_history['average_price'] = daily_result['average_price']

    except Exception as e:
        print(f"⚠️ {current_date.strftime('%Y-%m-%d')} 시뮬레이션 실패: {str(e)}")

# ✅ 결과 요약 출력
print("\n===== ✅ 누적 시뮬레이션 결과 요약 =====")
print(f"총 매수 횟수: {accumulated_trading_history['buy_count']}")
print(f"총 매도 횟수: {accumulated_trading_history['sell_count']}")
print(f"총 실현 손익: {accumulated_trading_history['realized_pnl']:.0f} KRW")
print(f"총 미실현 손익: {accumulated_trading_history['unrealized_pnl']:.0f} KRW")
print(f"최종 예수금: {accumulated_trading_history['initial_capital']:.0f} KRW")
print(f"보유 수량: {accumulated_trading_history['total_quantity']}")
print(f"평균 매수가: {accumulated_trading_history['average_price']:.2f} KRW")




실전투자 API 객체가 성공적으로 생성되었습니다.

📅 2023-01-01 시뮬레이션 중
ohlc : [[datetime.datetime(2022, 9, 23, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 77000.0, 77000.0, 74800.0, 74800.0, 13106.0], [datetime.datetime(2022, 9, 26, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 74200.0, 74500.0, 72000.0, 72200.0, 39425.0], [datetime.datetime(2022, 9, 27, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 72900.0, 73200.0, 71400.0, 72100.0, 33181.0], [datetime.datetime(2022, 9, 28, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 71300.0, 72200.0, 69800.0, 70300.0, 41416.0], [datetime.datetime(2022, 9, 29, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 70800.0, 71200.0, 69000.0, 69400.0, 41948.0], [datetime.datetime(2022, 9, 30, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 69000.0, 70300.0, 68800.0, 69500.0, 55859.0], [datetime.datetime(2022, 10, 4, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Seoul')), 69700.0, 71600.0, 69600.0, 70700.0, 23666.0], [datetime.datetime(2022, 10, 5, 0, 0, t

In [4]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time

id = 'bnuazz15'
trading_bot_name = 'test_bot'
auto_trading_stock = AutoTradingBot(id=id, virtual=True)

symbol = '207940'
symbol_name = '삼성바이오로직스'
start_date = date(2023, 3, 1)
end_date = date(2025, 4, 3)
interval = 'day'
max_allocation = 0.01
target_trade_value_krw = 1000000  # 매수 목표 거래 금액
buy_trading_logic = ['check_wick']
sell_trading_logic = ['check_wick']

auto_trading_stock.trade(
    trading_bot_name=trading_bot_name,
    buy_trading_logic=buy_trading_logic,
    sell_trading_logic=sell_trading_logic,
    symbol=symbol,
    symbol_name=symbol_name,
    start_date=start_date,
    end_date=end_date,
    target_trade_value_krw=target_trade_value_krw,
    interval=interval,
    max_allocation = max_allocation
)



모의투자 API 객체가 성공적으로 생성되었습니다.
시간: 2025-04-03 00:00:00+09:00, open_price: 1035000 KRW, low_price: 1035000 KRW, high_price: 1127000 KRW, close_price: 1113000 KRW, 볼린저 밴드 정보: 상단: 1116942 KRW, 중단: 1070850 KRW, 하단: 1024758 KRW
윗꼬리 = 14000.0, 아랫꼬리 = 0.0, body = 78000.0
아랫꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님
윗꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님, 종가가 시가보다 낮지 않음, 볼린저 밴드 상단 근처가 아님
check_wick 로직 buy_signal = False
시간: 2025-04-03 00:00:00+09:00, open_price: 1035000 KRW, low_price: 1035000 KRW, high_price: 1127000 KRW, close_price: 1113000 KRW, 볼린저 밴드 정보: 상단: 1116942 KRW, 중단: 1070850 KRW, 하단: 1024758 KRW
윗꼬리 = 14000.0, 아랫꼬리 = 0.0, body = 78000.0
아랫꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님
윗꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님, 종가가 시가보다 낮지 않음, 볼린저 밴드 상단 근처가 아님
check_wick 로직 sell_signal = False
마지막 직전 봉 : 78000.0. 양봉 : True, 음봉 : False


In [3]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time
from app.scheduler.auto_trading_scheduler import scheduled_trading_schedulerbot_task, scheduled_trading_bnuazz15_task, scheduled_trading


scheduled_trading(id = 'bnuazz15', virtual = True, trading_bot_name = 'bnuazz15bot')



모의투자 API 객체가 성공적으로 생성되었습니다.
- buy_trading_logic: ['ema_breakout_trading2', 'ema_breakout_trading', 'sma_breakout_trading', 'macd_trading', 'trend_entry_trading', 'bottom_rebound_trading', 'bollinger_band_trading'], sell_trading_logic : ['rsi_trading', 'macd_trading', 'top_reversal_sell_trading', 'downtrend_sell_trading', 'bollinger_band_trading']
[04/07 14:03:44] WARNING: API 호출 횟수를 초과하여 호출 유량 획득까지 대기합니다.
🗑️ 삭제된 종목: 025900
🗑️ 삭제된 종목: 025980
🗑️ 삭제된 종목: 114090
🗑️ 삭제된 종목: 141080
🗑️ 삭제된 종목: 225570
🗑️ 삭제된 종목: 241710
🗑️ 삭제된 종목: 263750
🗑️ 삭제된 종목: 326030
✅ 트랜잭션 성공: 1744002225858
[💾 잔고 저장] 025980
✅ 트랜잭션 성공: 1744002226513
[💾 잔고 저장] 114090
✅ 트랜잭션 성공: 1744002227170
[💾 잔고 저장] 241710
✅ 트랜잭션 성공: 1744002227801
[💾 잔고 저장] 263750


In [5]:
from app.utils.auto_trading_bot import AutoTradingBot

auto_trading_stock = AutoTradingBot(id = 'bnuazz15', virtual= True)
auto_trading_stock.get_holdings_with_details()

모의투자 API 객체가 성공적으로 생성되었습니다.
[04/07 13:56:57] WARNING: API 호출 횟수를 초과하여 호출 유량 획득까지 대기합니다.


[{'symbol': '025980',
  'symbol_name': '아난티',
  'market': 'KRX',
  'quantity': 323,
  'price': 6190,
  'amount': 1999370,
  'profit': 3580,
  'profit_rate': 0.17937758982658497},
 {'symbol': '114090',
  'symbol_name': 'GKL',
  'market': 'KRX',
  'quantity': 179,
  'price': 11130,
  'amount': 1992270,
  'profit': -540,
  'profit_rate': -0.02709741520767158},
 {'symbol': '241710',
  'symbol_name': '코스메카코리아',
  'market': 'KRX',
  'quantity': 20,
  'price': 42750,
  'amount': 855000,
  'profit': -121000,
  'profit_rate': -12.397540983606557},
 {'symbol': '263750',
  'symbol_name': '펄어비스',
  'market': 'KRX',
  'quantity': 30,
  'price': 30900,
  'amount': 927000,
  'profit': -65700,
  'profit_rate': -6.618313689936537}]